In [ ]:
import geopandas as gpd
import folium

# Load the GeoJSON
gdf = gpd.read_file("easternshore.geojson")

# Reproject to Web Mercator (EPSG:3857) for accurate centroid
gdf_proj = gdf.to_crs(epsg=3857)

# Calculate centroid in projected space
centroid_proj = gdf_proj.geometry.centroid.iloc[0]

# Transform centroid back to WGS84 (EPSG:4326) for folium
centroid_wgs = gpd.GeoSeries(centroid_proj, crs=3857).to_crs(epsg=4326).geometry.iloc[0]

# Create folium map centered at the correct location
m = folium.Map(location=[centroid_wgs.y, centroid_wgs.x], zoom_start=8)

# Add GeoJSON to the map (original data in EPSG:4326)
folium.GeoJson(gdf).add_to(m)

m

In [ ]:
tif_path = "data/sentinel_eastern_shore_2018-06-01_2018-06-30.tif"
tif_path = "data/landsat_eastern_shore_1989-03-01_1989-03-31.tif"

In [ ]:
import rasterio
import numpy as np

with rasterio.open(tif_path) as src:
    try:
        img = src.read(1)
        print(np.unique(img))
    except rasterio.errors.RasterioIOError as e:
        print(f"Failed to read block: {e}")

In [ ]:
from download_tools import get_mission, compute_ndwi, find_non_nan_window

In [ ]:
mission = get_mission("sentinel-2")
mission = get_mission("landsat-5")

In [ ]:
import matplotlib.pyplot as plt

band_indices = [mission['band_index']["green"], mission['band_index']['nir08']]  # e.g., green=3, nir=4 for landsat-5
([green_data, nir_data], profile, window) = find_non_nan_window(tif_path, bands=band_indices, window_size=800, stride=256)
window_data = green_data

if window_data is not None:
    plt.imshow(window_data, cmap="gray")
    plt.colorbar(label="Pixel Value")
    plt.title("Non-NaN Window Preview")
    plt.axis("off")
    plt.show()

In [ ]:
plt.imshow(green_data, cmap='viridis')
plt.title("Green Band")
plt.show()

plt.imshow(nir_data, cmap='viridis')
plt.title("NIR Band")
plt.show()

In [ ]:
compute_ndwi(green_data, nir_data, profile, out_path=None, display=True)

In [ ]:
from salinity_tools import extract_salinity_features_from_mosaic

In [ ]:
extract_salinity_features_from_mosaic(
    tif_path,
    mission["band_index"],
    "data/salinity_feature_mask.tif",
    "data/water_mask.tif",
    water_threshold=0.0,
)

In [ ]:
with rasterio.open("data/water_mask.tif") as src:
    water_mask = src.read(1)
    print("Total water pixels:", np.sum(water_mask > 0))

In [ ]:
window_data, profile, window = find_non_nan_window("data/water_mask.tif", window_size=250, stride=125)

if window_data is not None:
    plt.imshow(window_data, cmap="gray")
    plt.colorbar(label="Pixel Value")
    plt.title("Non-NaN Window Preview")
    plt.axis("off")
    plt.show()

print(np.sum(window_data))

In [ ]:
window_data, profile, window = find_non_nan_window("data/salinity_feature_mask.tif", window_size=100, stride=50)

if window_data is not None:
    plt.imshow(window_data, cmap="gray")
    plt.colorbar(label="Pixel Value")
    plt.title("Non-NaN Window Preview")
    plt.axis("off")
    plt.show()

print(np.sum(window_data))

In [ ]:
with rasterio.open("data/water_mask.tif") as src:
    try:
        img = src.read(1)
        print(src.height, src.width)
        plt.imshow(img, cmap="gray")
        plt.colorbar(label="Pixel Value")
        plt.title("Water Mask")
        plt.axis("off")
        plt.show()
        print(np.unique(img))
    except rasterio.errors.RasterioIOError as e:
        print(f"Failed to read block: {e}")

In [ ]:
with rasterio.open("data/salinity_feature_mask.tif") as src:
    try:
        img = src.read(1)
        print(src.height, src.width)
        plt.imshow(img, cmap="viridis")
        plt.colorbar(label="Pixel Value")
        plt.title("Non-NaN Window Preview")
        plt.axis("off")
        plt.show()
        print(np.unique(img))
    except rasterio.errors.RasterioIOError as e:
        print(f"Failed to read block: {e}")